Creating all pair in (shop_id, item_id) for each date_block_num. And if row has not item_count from origin sales data, item_count will fill 0.

In [22]:
# coding: utf-8
import os
from itertools import product

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from IPython.display import display

% matplotlib inline

## Config

In [23]:
# trn_path = './data/input/train.csv'
tst_path = './data/input/test.csv'
id_col = 'ID'
label_col = 'item_cnt_month'

submission_path = './data/output/submission/stacking_simple.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [24]:
item_cate_df = pd.read_csv('./data/input/item_categories.csv')
item_df = pd.read_csv('./data/input/items.csv')
sales_df = pd.read_csv('./data/input/sales_train.csv')
shop_df = pd.read_csv('./data/input/shops.csv')

In [25]:
sales_df['revenue'] = sales_df['item_price'] * sales_df['item_cnt_day']
trn_df = sales_df.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().reset_index()
trn_df = trn_df.rename(columns={'item_cnt_day': 'item_cnt_month'})
trn_df['ID'] = 0
# trn_df['ds_type'] = 'trn'

tst_df = pd.read_csv(tst_path)
tst_df['date_block_num'] = 34
tst_df['item_cnt_month'] = 0
# tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])
full_df.head()

,ID,date_block_num,item_cnt_month,item_id,shop_id
0,0,1,31.0,30,0
1,0,1,11.0,31,0
2,0,0,6.0,32,0
3,0,1,10.0,32,0
4,0,0,3.0,33,0


## Get grid

(shop_id, item_id) for each date_block_num

In [26]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = []
for block_num in full_df['date_block_num'].unique():
    cur_shops = full_df.loc[full_df['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = full_df.loc[full_df['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype='int32'))

grid = pd.DataFrame(np.vstack(grid), columns=index_cols, dtype=np.int32)

full_df = grid.merge(full_df, how='left', on=index_cols).fillna(0)

In [27]:
# for dbn in range(0, 34):
#     # create pair in (shop_id, item_id)
#     unique_shop_id = full_df[full_df['date_block_num'] == dbn]['shop_id'].copy().drop_duplicates()
#     unique_item_id = full_df[full_df['date_block_num'] == dbn]['item_id'].copy().drop_duplicates()
#     m_index = pd.MultiIndex.from_product([unique_shop_id, unique_item_id], names=['shop_id', 'item_id'])
#     val_df = pd.DataFrame([], index=m_index).reset_index()
#     val_df['ID'] = 0
#     val_df['date_block_num'] = dbn
# #     # distinguish origin train data
# #     val_df['ds_type'] = 'add_val'
#     # remove pair (shop_id, item_id) exists in full_df
#     origin_33_df = full_df[full_df['date_block_num'] == dbn][['item_id', 'shop_id', 'item_cnt_month']]
#     val_df = val_df.merge(origin_33_df, how='left', on=['item_id', 'shop_id'])
#     val_df = val_df[val_df.item_cnt_month.isnull()]
#     # no item_count then fill 0
#     val_df['item_cnt_month'] = 0

#     # merge to full_df
#     full_df = pd.concat([full_df, val_df])

In [28]:
full_df = full_df.sort_values(['date_block_num', 'shop_id', 'item_id'])

## Features

In [29]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])
    
def remove_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns = [f for f in feature_columns if f not in features]

**shop_id, item_id**

In [30]:
# add_features(['shop_id', 'item_id'])

**datetime info**

In [31]:
full_df['dt_year'] = full_df['date_block_num'] // 12 + 2013
full_df['dt_month'] = full_df['date_block_num'] % 12 + 1
add_features(['dt_year', 'dt_month'])

**item_df**

In [32]:
# item_name's words count
item_df['item_name_words_count'] = item_df['item_name'].map(lambda x: len(x.split(' ')))

# add to full_df
full_df = full_df.merge(item_df, how='left', on='item_id')

# add feautures
add_features(['item_category_id', 'item_name_words_count'])

In [33]:
sales_df = sales_df.merge(item_df, how='left', on='item_id')

**mean encoding**

In [34]:
# global_mean = full_df[full_df['date_block_num'] < 34]['item_cnt_month'].mean()

# def get_aggragation_feature(df, groupby_cols, agg_col, fillna_value):
#     gb = df.groupby(groupby_cols)[agg_col]
#     fname_fmt = '-'.join(groupby_cols+[agg_col]) + ':%s'
#     agg_df = pd.DataFrame({
#             fname_fmt%'mean': gb.mean(),
# #             fname_fmt%'median': gb.median(),
# #             fname_fmt%'max': gb.max(),
# #             fname_fmt%'min': gb.min(),
#         })
    
#     new_df = df.join(agg_df, on=groupby_cols).fillna(fillna_value)
#     return new_df, agg_df.columns.tolist()

# groupby_cols_list = [
#     ['shop_id', 'item_id'],
#     ['shop_id'],
#     ['item_id'],
# #     ['date_block_num'],
# #     ['dt_year'],
# #     ['dt_month'],
# #     ['item_category_id'],
# ]
# for groupby_cols in groupby_cols_list:
#     full_df, new_feats = get_aggragation_feature(full_df, groupby_cols, 'item_cnt_month', global_mean)
#     add_features(new_feats)

In [35]:
global_mean = full_df[full_df['date_block_num'] < 34]['item_cnt_month'].mean()

mean_encoding_cols_list = [
    ['shop_id', 'item_id'],
    ['shop_id'],
    ['item_id'],
]
get_mean_encoding_feat_name = lambda cols: 'mean_encoding-' + '-'.join(cols)

for cols in mean_encoding_cols_list:
    full_df[get_mean_encoding_feat_name(cols)] = np.nan
    add_features(get_mean_encoding_feat_name(cols))

kf = KFold(5, shuffle=True, random_state=20180717)
for trn_idx, val_idx in kf.split(full_df['item_cnt_month'].values):
    trn_df, val_df = full_df.iloc[trn_idx].copy(), full_df.iloc[val_idx].copy()
    for cols in mean_encoding_cols_list:
        feat_name = get_mean_encoding_feat_name(cols)
        mean_map = trn_df.groupby(cols)['item_cnt_month'].mean()
        val_df[feat_name] = val_df[cols].join(mean_map, how='left', on=cols)['item_cnt_month']
    full_df.iloc[val_idx] = val_df
full_df.fillna(global_mean, inplace=True)

**lag features**

In [36]:
# 获取当月的一些数据（然后进行lag）
cur_month_cols_list = [
    ['shop_id'],
    ['item_id'],
    ['shop_id', 'item_id'],
    ['item_category_id'],
    ['shop_id', 'item_category_id'],
]
cur_month_feats = []
for cols in cur_month_cols_list:
    print(cols)
    gb = sales_df.groupby(['date_block_num'] + cols) \
        .agg({'item_cnt_day': 'sum'}) \
        .rename(columns={'item_cnt_day': 'target-' + '-'.join(cols)})
    cur_month_feats.append('target-' + '-'.join(cols))
    full_df = full_df.join(gb, how='left', on=['date_block_num'] + cols).fillna(0)

['shop_id']
['item_id']
['shop_id', 'item_id']
['item_category_id']
['shop_id', 'item_category_id']


In [37]:
shift_range = [1,2,3,6,12]
shift_features = cur_month_feats

for shift_month in shift_range:
    print(shift_month)
    tmp_df = full_df[['shop_id', 'item_id', 'date_block_num'] + shift_features].copy()
    tmp_df['date_block_num'] = tmp_df['date_block_num'] + shift_month
    for f in shift_features:
        new_f = f + '_lag_' + str(shift_month)
        tmp_df = tmp_df.rename(columns={f: new_f})
        add_features(new_f)
    full_df = full_df.merge(tmp_df, how='left', on=['shop_id', 'item_id', 'date_block_num']).fillna(0)

1
2
3
6
12


In [38]:
# remove 2013
full_df = full_df[full_df['date_block_num'] > 12]

**show all features**

In [39]:
feature_columns

['dt_year',
 'dt_month',
 'item_category_id',
 'item_name_words_count',
 'mean_encoding-shop_id-item_id',
 'mean_encoding-shop_id',
 'mean_encoding-item_id',
 'target-shop_id_lag_1',
 'target-item_id_lag_1',
 'target-shop_id-item_id_lag_1',
 'target-item_category_id_lag_1',
 'target-shop_id-item_category_id_lag_1',
 'target-shop_id_lag_2',
 'target-item_id_lag_2',
 'target-shop_id-item_id_lag_2',
 'target-item_category_id_lag_2',
 'target-shop_id-item_category_id_lag_2',
 'target-shop_id_lag_3',
 'target-item_id_lag_3',
 'target-shop_id-item_id_lag_3',
 'target-item_category_id_lag_3',
 'target-shop_id-item_category_id_lag_3',
 'target-shop_id_lag_6',
 'target-item_id_lag_6',
 'target-shop_id-item_id_lag_6',
 'target-item_category_id_lag_6',
 'target-shop_id-item_category_id_lag_6',
 'target-shop_id_lag_12',
 'target-item_id_lag_12',
 'target-shop_id-item_id_lag_12',
 'target-item_category_id_lag_12',
 'target-shop_id-item_category_id_lag_12']

In [40]:
full_df

,shop_id,item_id,date_block_num,ID,item_cnt_month,dt_year,dt_month,item_name,item_category_id,item_name_words_count,...,target-shop_id_lag_6,target-item_id_lag_6,target-shop_id-item_id_lag_6,target-item_category_id_lag_6,target-shop_id-item_category_id_lag_6,target-shop_id_lag_12,target-item_id_lag_12,target-shop_id-item_id_lag_12,target-item_category_id_lag_12,target-shop_id-item_category_id_lag_12
4836102,2,30,13,0.0,0.0,2014,2,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,3,...,875.0,19.0,0.0,24130.0,63.0,488.0,861.0,0.0,31649.0,40.0
4836103,2,31,13,0.0,0.0,2014,2,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,4,...,875.0,25.0,0.0,8680.0,26.0,488.0,628.0,4.0,6307.0,21.0
4836104,2,32,13,0.0,0.0,2014,2,1+1,40,1,...,875.0,72.0,0.0,24130.0,63.0,488.0,208.0,0.0,31649.0,40.0
4836105,2,33,13,0.0,0.0,2014,2,1+1 (BD),37,2,...,875.0,35.0,0.0,8680.0,26.0,488.0,39.0,0.0,6307.0,21.0
4836106,2,34,13,0.0,0.0,2014,2,10 000 ЛЕТ ДО НАШЕЙ ЭРЫ WB (регион),40,8,...,875.0,12.0,0.0,24130.0,63.0,488.0,11.0,0.0,31649.0,40.0
4836107,2,36,13,0.0,0.0,2014,2,10 ЛЕТ СПУСТЯ (BD),37,4,...,875.0,3.0,0.0,8680.0,26.0,488.0,8.0,0.0,6307.0,21.0
4836108,2,37,13,0.0,0.0,2014,2,10 ЛЕТ СПУСТЯ (регион),40,4,...,875.0,15.0,0.0,24130.0,63.0,0.0,0.0,0.0,0.0,0.0
4836109,2,40,13,0.0,0.0,2014,2,100 Best classical melodies (mp3-CD) (Digipack),57,6,...,875.0,3.0,0.0,1316.0,0.0,488.0,5.0,0.0,1540.0,0.0
4836110,2,42,13,0.0,0.0,2014,2,100 Best romantic melodies (mp3-CD) (Digipack),57,6,...,875.0,4.0,0.0,1316.0,0.0,488.0,6.0,0.0,1540.0,0.0
4836111,2,44,13,0.0,0.0,2014,2,100 лучших мелодий против стресса (mp3-CD) (CD...,57,7,...,875.0,2.0,0.0,1316.0,0.0,488.0,5.0,0.0,1540.0,0.0


**clip target**

In [41]:
full_df['item_cnt_month'] = full_df['item_cnt_month'].clip(0, 20)

## Level1 Ensemble

**Valdation**

In [42]:
dates = full_df['date_block_num'].copy()

dates_trn = dates[dates <= 33]
dates_tst = dates[dates == 34]

trn_df = full_df[dates <= 33]
tst_df = full_df[dates == 34]

X_trn = trn_df[feature_columns]
y_trn = trn_df[label_col]
X_tst = tst_df[feature_columns]

**Score**

In [43]:
# score
def rmse(y, y_pred):
    return np.sqrt(metrics.mean_squared_error(y, y_pred))

**ElasticNet parameters tuning**

In [44]:
en_params = {
    'alpha': 0.01,
    'l1_ratio': 0.5,
}

**LightGBM parameters tuning**

In [45]:
lgb_params = {
    'feature_fraction': 0.75,
    'metric': 'rmse',
    'nthread': 1, 
    'min_data_in_leaf': 2**7, 
    'bagging_fraction': 0.75, 
    'learning_rate': 0.03, 
    'objective': 'mse', 
    'bagging_seed': 2**7, 
    'num_leaves': 2**7,
    'bagging_freq': 1,
    'verbose': 0,
    'max_depth': 5,
}

**Get level2 train data**

In [46]:
level2_block_num = [28, 29, 30, 31, 32, 33]
dates_trn_level2 = dates_trn[dates_trn.isin(level2_block_num)]
y_trn_level2 = y_trn[dates_trn.isin(level2_block_num)]

In [47]:
X_trn_level2 = np.zeros([y_trn_level2.shape[0], 2])

for cur_block_num in level2_block_num:
    print(cur_block_num)

    X_trn_i = X_trn[dates_trn < cur_block_num]
    y_trn_i = y_trn[dates_trn < cur_block_num]
    X_tst_i = X_trn[dates_trn == cur_block_num]
    
    en = ElasticNet(**en_params)
    en.fit(X_trn_i, y_trn_i)
    X_trn_level2[dates_trn_level2 == cur_block_num, 0] = en.predict(X_tst_i)
    
    lgb_model = lgb.train(lgb_params, lgb.Dataset(X_trn_i, label=y_trn_i), 100)
    X_trn_level2[dates_trn_level2 == cur_block_num, 1] = lgb_model.predict(X_tst_i)

28
29
30
31
32
33


**Get level2 test data**

In [48]:
en = ElasticNet(**en_params)
en.fit(X_trn, y_trn)
X_tst_level2_en = en.predict(X_tst)

lgb = lgb.train(lgb_params, lgb.Dataset(X_trn, label=y_trn), 100)
X_tst_level2_lgb = lgb.predict(X_tst)

X_tst_level2 = np.c_[X_tst_level2_en, X_tst_level2_lgb]

## Level2 Ensemble: Stacking

In [49]:
# XX_trn_level2 = X_trn_level2[dates_trn_level2 < 33]
# yy_trn_level2 = y_trn_level2[dates_trn_level2 < 33]
# XX_val_level2 = X_trn_level2[dates_trn_level2 == 33]
# yy_val_level2 = y_trn_level2[dates_trn_level2 == 33]

# lr = LinearRegression()
# lr.fit(XX_trn_level2, yy_trn_level2)

# rmse(yy_val_level2, lr.predict(XX_val_level2))

In [50]:
lr = LinearRegression()
lr.fit(X_trn_level2, y_trn_level2)
predictions = lr.predict(X_tst_level2)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [51]:
rmse(y_trn_level2, lr.predict(X_trn_level2).clip(0,20))

0.7845857538243391

## Predict

In [33]:
res_df = pd.DataFrame(predictions.clip(0,20), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].astype(int).values
res_df.sort_values('ID')[[output_id_col, output_label_col]].to_csv(submission_path, index=False)